**извлечение признаков из текста на естественном языке**

классификатор текстов лемматизация+TfidfVectorizer

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import gzip
import numpy as np
import numpy.random as rng
import pandas as pd
from tqdm import tqdm

np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# import numpy as np
# import pandas as pd
# import re

In [3]:
from sklearn import __version__ as SKLEARN_VERSION

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

SKLEARN_VERSION

'0.23.2'

In [4]:
from pymorphy2 import MorphAnalyzer

## тексты

In [5]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
112,"Положение команд после пятого тура Лиги чемпионов В пятом туре группового\nэтапа Лиги чемпионов ""Манчестер юнайтед"" не смог воспользоваться\nпреимуществом своего поля, сыграв вничью 0:0 с испански...",sport
1928,"Уходящий президент США Барак Обама арендовал офисное помещение в здании Всемирного фонда дикой природы (World Wildlife Fund, WWF), находящемся в центре Вашингтона, штат Колумбия. Об этом в понедел...",politics


## токенизация и очистка

In [6]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

In [7]:
# лемматизация и очистка с помощью пакета морфологического анализа

morph = MorphAnalyzer()


# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

# NOUN (существительное), VERB (глагол), ADJF (прилагательное)
def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''


def tokenize_normalize(s):
    return [ word_normalizer(w) for w in s.split(' ') if len(w)>1 ]

In [8]:
data['ctext'] = data['text'].progress_apply(string_normalizer).progress_apply( tokenize_normalize )

100%|██████████| 3196/3196 [01:11<00:00, 44.96it/s]


In [9]:
vcb0 =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb0)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 22123 слов


In [10]:
data['ctext'] = data['ctext'].apply( ' '.join  )

In [11]:
data.sample(10)

,text,tag,ctext
2812,"Монополии здесь не место: От мировых IT-гигантов требуют соблюдать правила\n\nНа российском рынке недопустима монополия мировых интернет-гигантов. Об этом говорят эксперты, которые комментируют ис...",tech,монополия место мировой требовать правило российский рынок монополия мировой эксперт который комментировать иск лаборатория касперский американский компания лаборатория касперский пояс...
2107,"Мадрид, , 10:06 — REGNUM Мадридский «Атлетико» с крупным счетом уступил «Вильярреалу» в матче пятнадцатого тура чемпионата Испании по футболу. Встреча, прошедшая 12 декабря на стадионе «Эль-Мадриг...",sport,мадрид мадридский крупный счёт уступить матч пятнадцать тур чемпионат испания футбол встреча декабрь стадион завершиться счёт победа жёлтый субмарина принести точный удар джонатан дос ...
1895,"Самолет немецкой авиационной компании Lufthansa был вынужден совершить посадку в аэропорту Нью-Йорка. Об этом сообщает телеканал АВС, давая ссылку на авиационные власти.\n\nLufthansaэкстренно сел ...",incident,самолёт немецкий авиационный компания быть посадка аэропорт нью-йорк сообщать телеканал ссылка авиационный власть сесть аэропорт нью-йорк сообщаться вынужденный посадка прийтись рейс ко...
196,"Ющенко выступает за создание национальной базы данных о количестве\nсирот Президент Украины Виктор Ющенко отмечает, что отсутствие национальной\nбазы данных о количестве детей-сирот и детей, лишен...",politics,ющенко выступать создание национальный база количество сирота президент украина виктор ющенко отмечать отсутствие национальный база количество ребёнок родительский опека осложнять формирова...
1449,"Венесуэла на 72 часа закрыла границу с Колумбией\n\nМЕХИКО, 13 дек – РИА Новости. Президент Венесуэлы Николас Мадуро распорядился закрыть границу с Колумбией на 72 часа для борьбы с финансовой маф...",politics,венесуэла час закрыть граница колумбия мехико дек риа новость президент венесуэла николас распорядиться граница колумбия час борьба финансовый мафия сообщение правительство венесуэла отмечать...
451,"Обама признал роль ошибок США в зарождении ИГИЛ\n\n7 декабря 2016 в 6:48\n\nРБК\n\nПрезидент США Барак Обама в своей речи о проблемах безопасности признал, что зарождению и росту террористической ...",politics,обама признать роль ошибка сша зарождение декабрь рбк президент сша барак обама свой речь проблема безопасность признать зарождение рост террористический организация исламский государство способ...
95,Проект Enlightenment может остаться без домена enlightenment.org Сайт\nоконного менеджера Enlightenment (enlightenment.org) уже более двух\nнедель не функционирует. Ben Rockwood поведал в своем бл...,tech,проект мочь домен сайт оконный менеджер неделя функционировать поведать свой блог история инцидент
2014,"(обновлено 10:04 13.12.2016 )\n\nМОСКВА, 13 дек - Р-Спорт. Южнокорейские скелетонисты намерены бойкотировать чемпионат мира-2017 по бобслею и скелетону в Сочи из-за допингового скандала, сообщает ...",sport,москва дек южнокорейский чемпионат бобслей скелетон сочи допинговый скандал сообщать агентство ссылка слово чиновник корейский федерация бобслей скелетон чемпионат мир бобслей скелето...
2231,Трагедия произошла 5 ноября. Мальчику стало плохо после школьных соревнований.\n\nВ Красноярской школе на церемонии награждения победителей школьных соревнований по баскетболу 16-летнему мальчику ...,incident,трагедия произойти ноябрь мальчик стать школьный соревнование красноярский школа церемония награждение победитель школьный соревнование баскетбол мальчик стать больница скончаться уголовный...
1332,iPhone 7 Plus против Huawei Mate 9: фотобитва флагманов с двойными камерами\n\n4 декабря 2016 в 9:11\n\nДмитрий Смирнов / Фото: Дмитрий Брушко\n\n2016 год стал годом возвращения двойных камер в см...,tech,флагман двойной камера декабрь дмитрий смирнов фото дмитрий год стать год возвращение двойной камера смартфон стать флагманский гаджет простой устройство следующий год ожид...


##  Vectorizer

In [ ]:
def tokenizer(s): 
    for w in s.split(' '):
        if (len(w)>1): 
            yield w
            
tf = TfidfVectorizer(use_idf=True, norm='l2', tokenizer=tokenizer, token_pattern=None).fit( data['ctext'] )
vcb1 = sorted(tf.vocabulary_)
print(len(vcb1))
# pd.DataFrame( vcb1 ).to_csv('voc1.txt',index=False,header=False)

In [ ]:
# разница между исходным словарём и словарём векторайзера
set(vcb0)^set(vcb1)

## формируем датасеты

In [ ]:
X = tf.transform( data['ctext'] )
X.shape

In [ ]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

In [ ]:
y = data['tag'].map(labels).values
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=326 )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## обучаем классификатор

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000, tol=0.19)
clf.fit(X_train,y_train)

## тестируем

In [ ]:
# доля правильных ответов на учебном наборе
o = clf.predict(X_train)
accuracy_score(y_train,o)

In [ ]:
# доля правильных ответов на тестовом наборе
o = clf.predict(X_test)
accuracy_score(y_test,o)

In [ ]:
print( classification_report(y_test,o) )

---

In [ ]:
from matplotlib import pyplot as plt
import itertools

classes = sorted(labels.keys())
cm = confusion_matrix(y_test,o)
tick_marks = np.arange(len(classes))

plt.figure(figsize=(10,9))

plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.title('Confusion matrix')
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

plt.show()

---

In [ ]:
o = clf.predict(X)

labels_inv = { labels[k]:k for k in labels }
# labels_inv

for n in range(10):
    i = rng.randint(len(data))
    print('tag:',data.iloc[i,1])
    print('predict:',labels_inv[o[i]])
    print(re.sub(r'\n\s*\n',' ',data.iloc[i,0][:200])+'...')
    print('- - - - - - - - - - - - - - - - - -')
    

In [ ]:
# https://habr.com/ru/post/205360/